In [1]:
%matplotlib notebook
import numpy as np
import gdcm
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import sys
import dicom

In [32]:
def gdcm_to_numpy(image):
    """ Converts a GDCM image to a numpy array.
    :param image: GDCM.ImageReader.GetImage()
    
    From: https://github.com/neurosnap/mudicom/blob/master/mudicom/image.py
    """
    gdcm_typemap = {
        gdcm.PixelFormat.INT8:     np.int8,
        gdcm.PixelFormat.UINT8:    np.uint8,
        gdcm.PixelFormat.UINT16:   np.uint16,
        gdcm.PixelFormat.INT16:    np.int16,
        gdcm.PixelFormat.UINT32:   np.uint32,
        gdcm.PixelFormat.INT32:    np.int32,
        gdcm.PixelFormat.FLOAT32:  np.float32,
        gdcm.PixelFormat.FLOAT64:  np.float64
    }
    pixel_format = image.GetPixelFormat().GetScalarType()
    if pixel_format in gdcm_typemap:
        data_type = gdcm_typemap[pixel_format]
    else:
        raise KeyError('{} is not a supported pixel format'.format(pixel_format))
    
    dimensions = image.GetDimension(1), image.GetDimension(0)
    gdcm_array = image.GetBuffer()
    
    # if GDCM indicates that a byte swap is in order, make sure to inform numpy as well
    if image.GetNeedByteSwap():
        data_type.newbyteorder('S')

    # GDCM returns char* as type str. This converts it to type bytes
    if sys.version_info >= (3, 0):
        gdcm_array = gdcm_array.encode(sys.getfilesystemencoding(), "surrogateescape")

    # use float for accurate scaling
    dimensions = image.GetDimensions()
    result = np.frombuffer(gdcm_array, dtype=data_type)
    
    # FIXME: compare with pydicom's reshaping strategy
    # https://github.com/darcymason/pydicom/blob/b440b5aa674cbffa47c083420fb41069d1487d93/pydicom/dataset.py#L403-L422
    if len(dimensions) == 3:
        # for cine (animations) there are 3 dims: x, y, number of frames
        result.shape = dimensions[2], dimensions[0], dimensions[1]
    else:
        result.shape = dimensions
    return result

def get_pixel_data(filepath):
    """ Grabs image data and converts it to a numpy array """
    image_reader = gdcm.ImageReader()
    image_reader.SetFileName(filepath)
    if not image_reader.Read():
        raise IOError("Could not read DICOM image")
    pixel_array = gdcm_to_numpy(image_reader.GetImage())
    return pixel_array

In [35]:
filenames = [r"dicom\compressed_ct\EE0BC09D", 
             r"dicom\compressed_ct\EE0D46CA",
             r"dicom\compressed_mr\IM-0001-0001.dcm",
             r"dicom\compressed_mr\IM-0001-0002.dcm",
             r"dicom\oai_xrays\001",
             r"dicom\oai_xrays\002"]

In [36]:
for filename in filenames:
    fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
    fig.suptitle(filename)
    
    # GDCM
    ax1.set_title('GDCM')
    pixel_array = get_pixel_data(filename)
    ax1.imshow(pixel_array.T, cmap=cm.gray)

    # pydicom
    ax2.set_title('pydicom')
    try:
        dicom_file = dicom.read_file(filename)
        ax2.imshow(dicom_file.pixel_array.T, cmap=cm.gray)
    except NotImplementedError as e:
        ax2.set_axis_bgcolor('red')
        